In [49]:
import sync_wandb_run
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import utils as pu
from plot import create_plot_with_values
from plot_pretty import create_ultimate_plotly_plot, generate_scatter_plot, generate_scatter_plot_accuracy, generate_scatter_plot_gridscore
import plotly_utils

In [50]:
entity = "raphaelmaser"  # Your W&B username or team
project = "Improved-Interpretability-and-Concepts"  # Your W&B project name
df = sync_wandb_run.load_wandb_runs()
df.count()  # Display the number of runs loaded
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

val_accuracy = "Validation Accuracy"
grid_score = "PG Score"
frozen_layers = "config_define_frozen_layers"
trainable_params = "Trainable Parameters (M)"
time="Runtime (h)"
figsize=(10,4)


# Comparison between different LoRA styles

1. LORA applied to all matrices + ReLU
2. LoRA only applied to qkv projection without ReLU

WD=0.1 for all

In [51]:

vitc_lora_qkv = df[df['group'].isin(["vitc-lora-ranks"])] 
vitc_lora = df[df['group'].isin(["vit-conv-stem-lora"])]
vitc_lora["config_weight_decay"] = vitc_lora["config_weight_decay"].astype(float)  # Ensure weight decay is float type
vitc_lora = vitc_lora[vitc_lora["config_weight_decay"] == 0.1]  # Filter for weight decay 0.1
vitc_lora = vitc_lora[vitc_lora["config_lora_depth"] == 0]  # Filter for weight decay 0.1

vit_base = df[df['id'] == "2fupnc3h"] # Base model
vit_conv_stem_only = df[df['id'] == "uaryeifu"]  # Conv stem only model
vitc_base = df[df['id'] == "n8dqbqan"]  # Conv stem only model
vitc = pd.concat([vit_base, vit_conv_stem_only, vitc_base, vitc_lora_qkv, vitc_lora], ignore_index=True)  # Combine base and conv stem only models

vitc.reset_index(drop=True, inplace=True)  # Reset index after filtering
print(f"Number of runs in layerwise_freezing group: {len(vitc)}")

#Renaming
vitc = pu.rename(vitc).sort_values(by=val_accuracy, ascending=False)
vitc = pu.rename_adapter_factor_experiments(vitc)  # Rename columns for adapter factor experiments
vitc.rename(columns={
    "config_lora_rank": "Lora Rank",
    "config_trainable_parameters": trainable_params,
    "summary__runtime": time,
}, inplace=True)

vitc.replace({
    "Model": {
        "vitc_b_16_B=2_pretrained_lora_kaiming": "ViTc LoRA (No ReLU, only qkv)",
        "vitc_b_16_B=2_pretrained_dora_kaiming": "ViTc DoRA (No ReLU, only qkv)",
        "vitc_b_16_B=2_pretrained_lora_relu_kaiming": "ViTc LoRA (ReLU)",
        "vitc_b_16_B=2_pretrained_only-conv-stem": "ViTc - conv stem only",
        "vit_b_16_B=2_pretrained": "ViT",
        "vit_b_16_B=2_pretrained_conv-stem": "ViTc"
    }}, inplace=True)  # Rename specific model names for clarity

vitc[time] = vitc[time].apply(lambda x: x/(60*60))  # Format runtime to 2 decimal places
# Some stats
for name in vitc["Model"].unique(): print(name)  # Print each model name
display(vitc)


/tmp/ipykernel_3328953/1780748447.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Number of runs in layerwise_freezing group: 14
ViTc
ViTc LoRA (ReLU)
ViTc LoRA (No ReLU, only qkv)
ViTc - conv stem only
ViT


id                          Model     state        entity  \
2   n8dqbqan                           ViTc  finished  raphaelmaser   
11  1zn8qgax               ViTc LoRA (ReLU)  finished  raphaelmaser   
12  ujensn29               ViTc LoRA (ReLU)  finished  raphaelmaser   
13  sdfqpo0q               ViTc LoRA (ReLU)  finished  raphaelmaser   
3   3m2yljdz  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
9   ck8pz3ho               ViTc LoRA (ReLU)  finished  raphaelmaser   
7   fphpl0nv  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
8   4g0k0tjg  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
5   szxir6fn  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
10  06xy8vza               ViTc LoRA (ReLU)  finished  raphaelmaser   
4   x0rv38z9  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
6   p0y0kg5c  ViTc LoRA (No ReLU, only qkv)  finished  raphaelmaser   
1   uaryeifu          ViTc - conv stem only  finished  raphaelmaser   
0   2fupnc3h                            ViT  finished  raphaelmaser   

                                   project            created_at  updated_at  \
2   Improved-Interpretability-and-Concepts  2025-07-19T06:49:42Z         NaN   
11  Improved-Interpretability-and-Concepts  2025-07-19T06:56:45Z         NaN   
12  Improved-Interpretability-and-Concepts  2025-07-20T08:46:53Z         NaN   
13  Improved-Interpretability-and-Concepts  2025-07-20T08:46:29Z         NaN   
3   Improved-Interpretability-and-Concepts  2025-07-29T22:46:34Z         NaN   
9   Improved-Interpretability-and-Concepts  2025-07-19T21:35:35Z         NaN   
7   Improved-Interpretability-and-Concepts  2025-07-29T21:40:31Z         NaN   
8   Improved-Interpretability-and-Concepts  2025-07-29T21:01:04Z         NaN   
5   Improved-Interpretability-and-Concepts  2025-07-29T21:01:04Z         NaN   
10  Improved-Interpretability-and-Concepts  2025-07-19T22:40:52Z         NaN   
4   Improved-Interpretability-and-Concepts  2025-07-29T20:59:28Z         NaN   
6   Improved-Interpretability-and-Concepts  2025-07-29T20:51:04Z         NaN   
1   Improved-Interpretability-and-Concepts  2025-07-21T10:28:29Z         NaN   
0   Improved-Interpretability-and-Concepts  2025-07-19T06:48:52Z         NaN   

   tags notes                                                url  \
2   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
11  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
12  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
13  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
3   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
9   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
7   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
8   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
5   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
10  NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
4   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
6   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
1   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   
0   NaN   NaN  https://wandb.ai/raphaelmaser/Improved-Interpr...   

                 group  job_type  sweep                   synced_at  config_b  \
2        vit-conv-stem       NaN    NaN  2025-07-27T10:49:36.411784       2.0   
11  vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:38.005564       2.0   
12  vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:41.667236       2.0   
13  vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:40.758101       2.0   
3      vitc-lora-ranks       NaN    NaN  2025-07-30T13:42:45.370527       2.0   
9   vit-conv-stem-lora       NaN    NaN  2025-07-27T10:49:38.970541       2.0   
7      vitc-lora-ranks       NaN    NaN  2025-07-30T13:42:44.474280       2.0   
8      vitc-lora-ranks       NaN    NaN  2025-07-30T13:42:42.646008       

## Weight decay = 0.1

In [52]:

generate_scatter_plot_accuracy(
    data=vitc,
    x=trainable_params,
    y=val_accuracy,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [53]:
generate_scatter_plot_gridscore(
    data=vitc,
    x=trainable_params,
    y=grid_score,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params
)

In [54]:

generate_scatter_plot_accuracy(
    data=vitc,
    x=trainable_params,
    y=time,
    color="Model",
    scale_factor=2,
    title="ViTc - With Weight Decay (0.1)",
    connect_dots=True,
    scale_dots_by_trainable_params=trainable_params,
    range_y=(0, 7)
)